In [7]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import requests
import pandas as pd


## TASK 1
Write a fucntion that loads series **SG_GEN_PARL** from the UN SDG API and returns a pandas dataframe with the two columns **year** and **prop_seats_women**. The input is **geoAreaCode**.

* API documentation: https://unstats.un.org/sdgapi/swagger/ 
* Relevant notebook from lecture 4: https://github.com/gochristoph/big-data-for-public-policy-2023/blob/main/lecture_04/lecture_4_with_solutions.ipynb

In [8]:
def load_SG_GEN_PARL(geoAreaCode):
    assert isinstance(geoAreaCode, int)
    url = f'https://unstats.un.org/sdgapi/v1/sdg/Series/SG_GEN_PARL/GeoArea/{geoAreaCode}/DataSlice'
    ...
    return pd.DataFrame(data={'year': year, 'prop_seats_women': prop_seats_women})

In [9]:
# testing the function
display(load_SG_GEN_PARL(1).head())

,year,prop_seats_women
0,2003,15.12
1,2007,17.14
2,2004,15.21
3,2005,15.91
4,2001,13.79


## TASK 2 

Write a function that gets all geoAreaCodes from the UN SDG API and converts them to a valid `options` input for `dcc.Dropdown` (a list of dictionaries with the keys `label` and `value` for each dictionary in the list).

For the API call, take a look at: https://unstats.un.org/sdgapi/swagger/#!/GeoArea/V1SdgGeoAreaListGet

Example for context:

```
dcc.Dropdown(
   options=[
       {'label': 'World', 'value': 1},
       {'label': 'Switzerland', 'value': 756},
       ...
   ],
   value=1
)
```

In [10]:
def load_geoArea_options():
    url = f'https://unstats.un.org/sdgapi/v1/sdg/GeoArea/List'
    response = requests.get(url)
    options = response.json()
    ...
    return options

geoArea_options = load_geoArea_options()

In [11]:
# testing the function and creating the options
geoArea_options = load_geoArea_options()
geoArea_options[:10]

[{'label': 'Afghanistan', 'value': '4'},
 {'label': 'Åland Islands', 'value': '248'},
 {'label': 'Albania', 'value': '8'},
 {'label': 'Algeria', 'value': '12'},
 {'label': 'American Samoa', 'value': '16'},
 {'label': 'Andorra', 'value': '20'},
 {'label': 'Angola', 'value': '24'},
 {'label': 'Anguilla', 'value': '660'},
 {'label': 'Antarctica', 'value': '10'},
 {'label': 'Antigua and Barbuda', 'value': '28'}]

## TASK 3
Have fun an run **your own web app**!

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='My Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(options=geoArea_options, 
                 value='1', 
                 id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    df = load_SG_GEN_PARL(geoAreaCode=int(value))
    return px.scatter(df, x='year', y='prop_seats_women')

if __name__ == '__main__':
    app.run_server(debug=False, port=8049, use_reloader=False)